In [1]:
# Importando as bibliotecas que serão utilizadas
import mysql.connector
import warnings
from mysql.connector import Error
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [2]:
# Funções auxiliares para facilitar e compactar o código

# Conectar ao banco de dados

def connect_db():
    return mysql.connector.connect(
        host = '127.0.0.1', 
        user = 'root', 
        password = 'Rafinha24.1_', 
        database = 'Locadora_Veiculos'
    )

# Executar Querys

def execute_query(query, params=None):
    conn = connect_db()
    if conn is not None:
        cursor = conn.cursor()
        try:
            cursor.execute(query, params)
            conn.commit()
        finally:
            cursor.close()
            conn.close()

def fetch_query(query, params=None):
    conn = connect_db()
    if conn is not None:
        cursor = conn.cursor()
        try:
            cursor.execute(query, params)
            return cursor.fetchall()
        finally:
            cursor.close()
            conn.close()

def fetch_query_grafs(query, params=None):
    conn = connect_db()
    cursor = conn.cursor()
    
    cursor.execute(query, params)
    results = cursor.fetchall()
    
    cursor.close()
    conn.close()
    
    return results

# Verificar cadastros prévios para chaves estrangeiras

def verificar_cpf_existe(cpf):
    query = "SELECT 1 FROM pessoa_fisica WHERE CPF = %s"
    result = fetch_query(query, (cpf,))
    return len(result) > 0

def verificar_cnpj_existe(cnpj):
    query = "SELECT 1 FROM pessoa_juridica WHERE CNPJ = %s"
    result = fetch_query(query, (cnpj,))
    return len(result) > 0

def verificar_placa_existe(placa):
    query = "SELECT 1 FROM veiculo WHERE Placa = %s"
    result = fetch_query(query, (placa,))
    return len(result) > 0

def verificar_reserva_existe(num_reserv):
    query = "SELECT 1 FROM subsis_locacao WHERE Num_reserv = %s"
    result = fetch_query(query, (num_reserv,))
    return len(result) > 0

def verificar_consul_existe(num_consul):
    query = "SELECT 1 FROM subsis_reserv WHERE Num_consul = %s"
    result = fetch_query(query, (num_consul,))
    return len(result) > 0

def verificar_conta_existe(conta):
    query = "SELECT 1 FROM locatario WHERE Conta = %s"
    result = fetch_query(query, (conta,))
    return len(result) > 0

def verificar_patio_existe(patio):
    query = "SELECT 1 FROM subsis_locacao WHERE Patio_entra = %s OR Patio_saida = %s"
    result = fetch_query(query, (patio, patio))
    return len(result) > 0

def verificar_modelo_existe(modelo):
    query = "SELECT 1 FROM veiculo WHERE Modelo = %s"
    result = fetch_query(query, (modelo,))
    return len(result) > 0

def verificar_marca_existe(marca):
    query = "SELECT 1 FROM veiculo WHERE Marca = %s"
    result = fetch_query(query, (marca,))
    return len(result) > 0

In [3]:
# Definindo as funções a serem executadas pela aplicação

# Módulo 1: Realizar Cadastro, Atualização ou Reserva

# 1. Cadastro/Atualização de Clientes
    
# 1.1 Cadastro de PF
        
def cadastrar_cliente_pf():
    
    # Obtendo inputs
    CPF = input("Insira o CPF do cliente (ex.:111.111.111-11: ")
    
    # Verificar se o CPF existe
    if CPF and verificar_cpf_existe(CPF):
        print("\nUma pessoa com esse CPF já existe.")
        return
    
    Nasc = input("Insira a data de nascimento do cliente (ex.: AAAA-MM-DD): ")
    Nome = input("Insira o nome do cliente: ")
    
    query = "INSERT INTO pessoa_fisica (CPF, Nasc, Nome) VALUES (%s, %s, %s)"
    params = (CPF, Nasc, Nome)
    
    execute_query(query, params)
    print("\nCliente cadastrado com sucesso!\n")
    
# 1.2 Cadastro de PJ
        
def cadastrar_cliente_pj():
    
    # Obtendo inputs
    CNPJ = input("Insira o CNPJ da empresa (ex.:11.111.111/1111-11: ")
    
    # Verificar se o CNPJ existe
    if CNPJ and verificar_cnpj_existe(CNPJ):
        print("\nUma empresa com esse CNPJ já existe.")
        return
    
    Nome_Fantasia = input("Insira o nome da empresa: ")
    
    query = "INSERT INTO pessoa_juridica (CNPJ, Nome_Fantasia) VALUES (%s, %s)"
    params = (CNPJ, Nome_Fantasia)
    
    execute_query(query, params)
    print("\nEmpresa cadastrada com sucesso!\n")
    
def cadastrar_funcionario():
    
    # Obtendo inputs
    Reg_func = input("Insira o registro do funcionário (ex.: F001): ")
    CPF = input("Insira o CPF do funcionário (ex.: 123.456.789-00): ")
    
    # Verificar se o CPF existe
    if CPF and not verificar_cpf_existe(CPF):
        print("\nCPF não existe, cadastre a pessoa física.")
        return
    
    CNPJ = input("Insira o CNPJ da empresa (ex.: 00.123.456/0001-00): ")
    
    # Verificar se o CNPJ existe
    if CNPJ and not verificar_cnpj_existe(CNPJ):
        print("\nCNPJ não existe, cadastre a pessoa jurídica.")
        return

    query = "INSERT INTO funcionario (Reg_func, CPF, CNPJ) VALUES (%s, %s, %s)"
    params = (Reg_func, CPF, CNPJ)
    
    execute_query(query, params)
    print("\nFuncionário cadastrado com sucesso!\n")

    
# 1.3 Cadastro de Locatario
        
def cadastrar_locatario():
    
    # Obtendo inputs
    Conta = input("Insira a conta do locatário (ex.:123456789-0: ")
    CPF = input("Insira o CPF do cliente (ex.:111.111.111-11: ")
    
    # Verificar se o CPF existe
    if CPF and not verificar_cpf_existe(CPF):
        print("\nCPF não existe, cadastre a pessoa física.")
        return
    
    CNPJ = input("Insira o CNPJ da empresa (ex.:11.111.111/1111-11: ")
    
    # Verificar se o CNPJ existe
    if CNPJ and not verificar_cnpj_existe(CNPJ):
        print("\nCNPJ não existe, cadastre a pessoa jurídica.")
        return

    Agencia = input("Insira a agência do locatário (ex.: Agência A: ")
    
    query = "INSERT INTO locatario (Conta, CPF, CNPJ, Agencia) VALUES (%s, %s, %s, %s)"
    params = (Conta, CPF, CNPJ, Agencia)
    
    execute_query(query, params)
    print("\nLocatário cadastrado com sucesso!\n")

# 1.4 Cadastro de Condutor
        
def cadastrar_condutor():
    
    # Obtendo inputs
    CNH = input("Insira a CNH do condutor (ex.:11111111111: ")
    CPF = input("Insira o CPF do condutor (ex.:111.111.111-11: ")
    
    # Verificar se o CPF existe
    if CPF and not verificar_cpf_existe(CPF):
        print("\nCPF não existe, cadastre a pessoa física.")
        return
    
    Placa = input("Insira a placa do veículo relacionado ao condutor (ex.: AAA1A11 ou AAA1111): ")
    
    # Verificar se a Placa existe
    if Placa and not verificar_placa_existe(Placa):
        print("\nA placa não existe, cadastre o veículo.")
        return
    
    Conta = input("Insira a conta do locatário relacionado ao condutor (ex.:123456789-0: ")
    
    # Verificar se a Conta existe
    if Conta and not verificar_placa_existe(Conta):
        print("\nA conta não existe, cadastre o locatário.")
        return
    
    Venc_CNH = input("Insira a data de Vencimento da CNH do condutor (ex.: AAAA-MM-DD): ")
    Cond_Princ = input("Insira 0. Não é o condutor principal do veículo \nou 1. É o condutor principal do veículo")
    
    query = "INSERT INTO condutor (CNH, CPF, Placa, Conta, Venc_CNH, Cond_Princ) VALUES (%s, %s, %s, %s, %s, %s)"
    params = (CNH, CPF, Placa, Conta, Venc_CNH, Cond_Princ)
    
    execute_query(query, params)
    print("\nCondutor cadastrado com sucesso!\n")

# 2. Cadastro/Atualização da Frota
        
# 2.1 Cadastro de veículo
        
def cadastrar_veiculo():
    
    # Obtendo inputs
    Placa = input("Insira a placa do veículo (ex.: AAA1A11 ou AAA1111): ")
    
    # Verificar se a Placa existe
    if Placa and verificar_placa_existe(Placa):
        print("\nUm veículo com essa placa já existe.")
        return
    
    Chassi = input("Insira o chassi do veículo (ex.:123456789ABCDEF01): ")
    Dimensoes = input("Insira a dimensão do veículo(ex.: 1.1m x 1.1m x 1.1m): ")
    
    # Solicitar a categoria com validação
    Categorias = ['Utilitário', 'Passeio', 'Comercial', 'Esportivo', 'SUV', 'Outro']
    while True:
        Categoria = input("Insira a categoria do veículo entre: Utilitário, Passeio, Comercial, Esportivo, SUV, Outro: ")
        if Categoria in Categorias:
            break
        else:
            print("\nCategoria inválida! Tente novamente uma das opções listadas.\n")
            
    Marca = input("Insira a marca do veículo: ")
    Cor = input("Insira a cor do veículo (ex.: preto, vermelho, verde): ")
    Modelo = input("Insira o modelo do veículo: ")
    Bebe_confort = input("Insira 0. Não possui Bebê Conforto \nou 1. Possui Bebê Conforto")
    Ar_cond = input("Insira 0. Não possui Ar Condicionado \nou 1. Possui Ar Condicionado")
    Mecaniz = input("Insira 0. Câmbio manual \nou 1. Câmbio automático")
    Preco = input("Insira o valor do aluguel (diária) do veículo: ")
    
    query = """INSERT INTO veiculo (Placa, Chassi, Dimensoes, Categoria, Marca, Cor,
                                    Modelo, Bebe_confort, Ar_cond, Mecaniz, Preco) 
                                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""  
    params = (Placa, Chassi, Dimensoes, Categoria, Marca, Cor, Modelo, Bebe_confort, Ar_cond, Mecaniz, Preco)

    execute_query(query, params)
    print("\nVeículo cadastrado com sucesso!\n")

# 2.2 Atualização do prontuário do veículo

def atualizar_cadastrar_prontuario():
    
    # Obtendo inputs
    Placa = input("Insira a placa do veículo (ex.: AAA1A11 ou AAA1111): ")
    
    # Verificar se a Placa existe
    if Placa and not verificar_placa_existe(Placa):
        print("\nA placa não existe, cadastre o veículo após o cadastramento do prontuário.")
    
    Quilometragem = input("Insira a quilometragem atual do veículo: ")
    Revisao = input("Insira 0. Não está com a revisão em dia \nou 1. Está com a revisão em dia")
    Idade_vei = input("Insira a idade atual do veículo: ")
    
    # Solicitar o Estado de conservação
    estados = ['Novo','Bom','Regular','Ruim']
    while True:
        Est_conserv = input("Insira o estado do veículo entre: Novo, Bom, Regular, Ruim: ")
        if Est_conserv in estados:
            break
        else:
            print("\nEstado inválida! Tente novamente uma das opções listadas.\n")
            
    Press_pneu = input("Insira a pressão atual do pneu do veículo: ")
    Niv_oleo = input("Insira o nível atual de óleo do veículo: ")
    
    query = """
        UPDATE prontuario
        SET Quilometragem = %s,
            Revisao = %s,
            Idade_vei = %s,
            Est_conserv = %s,
            Press_pneu = %s,
            Niv_oleo = %s
        WHERE Placa = %s
        """
    params = (Quilometragem, Revisao, Idade_vei, Est_conserv, Press_pneu, Niv_oleo, Placa)
    
    execute_query(query, params)
    print("\nProntuário atualizado com sucesso!\n")

# 2.3 Cadastro de foto do veículo

def cadastrar_foto_veiculo():
    
    # Obtendo inputs
    Placa = input("Insira a placa do veículo (ex.: AAA1A11 ou AAA1111): ")
    
    # Verificar se a Placa existe
    if Placa and not verificar_placa_existe(Placa):
        print("\nA placa não existe, cadastre o veículo.")
        return
    
   # Abrir uma janela de diálogo para selecionar o arquivo
    root = Tk()
    root.withdraw() # Esconder a janela principal do Tkinter
    caminho_foto = filedialog.askopenfilename(title = "Selecione a foto do veículo",
                                              filetypes = [("Arquivo de imagens", "*.jpg, *.jpeg, *.png, *.gif, *.bmp")])

    if not caminho_foto:
        print("\nNenhum arquivo selecionado.")
        return

    # Ler a imagem como binário
    try:
        with open(caminho_foto, 'rb') as file:
            Foto = file.read()
            
    except FileNotFoundError:
        print("\nArquivo não encontrado. Verifique o caminho e tente novamente.")
        return

    # Inserir a foto no banco de dados
    query = "INSERT INTO foto_ilus (Placa, Foto) VALUES (%s, %s)"
    params = (Placa, Foto)
    
    execute_query(query, params)
    print("\nFoto cadastrada com sucesso!\n")
    
# 3. Reserva de Veículo

# 3.1. Relatório de pátio

def relatorio_patio():
    
    # Verificando a presença de um Patio válido
    while True:
        # Obtendo inputs
        patio_selecionado = input("Insira o Pátio a ser pesquisado (ex.: Pátio A): ")

        # Verificar se a reserva existe
        if not verificar_patio_existe(patio_selecionado):
            print("\nO Patio não existe. Por favor, insira um Patio válido.")
        else:
            break
    
    # Consulta SQL
    query = """
    SELECT
        Num_reserv,
        Placa,
        Data_retirada_prev,
        Data_devol_prev,
        CASE
            WHEN Patio_entra = %s THEN 'Entrada'
            WHEN Patio_saida = %s THEN 'Saída'
            ELSE 'Indefinido'
        END AS Tipo_Patio
    FROM
        subsis_locacao
    WHERE
        Patio_entra = %s OR Patio_saida = %s;
    """
    params = (patio_selecionado, patio_selecionado, patio_selecionado, patio_selecionado)
    
    results = fetch_query(query, params)

    # Imprimir os resultados
    if results:
        for row in results:
            print(f"\nNúmero da reserva: {row[0]}, \nPlaca do veículo: {row[1]}",
                  f"\nData de retirada prevista: {row[2]}, \nData da devolução prevista: {row[3]}",
                  f"\nSituação: Pátio de {row[4]}\n")

# 3.2. Relatório de locação
 
def relatorio_locacao():
    
    # Verificando a presença de um CPF válido
    while True:
        # Obtendo inputs
        CPF = input("Insira o CPF da consulta para o relatório da locação: ")

        # Verificar se a reserva existe
        if not verificar_cpf_existe(CPF):
            print("\nO CPF não existe. Por favor, insira um CPF válido ou cadastre a pessoa física.")
        else:
            break
    
    # Consulta SQL
    query = """
    SELECT
        sl.CPF,
        pf.Nome,
        sl.Placa,
        sl.Num_reserv,
        sl.Data_retirada_prev,
        sl.Data_retirada_efet,
        sl.Data_compra,
        sl.Data_devol_prev,
        sl.Data_devol_efet
    FROM
        subsis_locacao sl
    INNER JOIN pessoa_fisica as pf
    ON sl.CPF = pf.CPF
    WHERE sl.CPF = %s;
    """

    results = fetch_query(query, (CPF,))

    # Imprimir os resultados
    if results:
        for row in results:
            print(f"\nCPF do locatario: {row[0]}, \nNome do locatario: {row[1]}",
                  f"\nPlaca do veículo alugado: {row[2]}, \nNúmero da reserva: {row[3]}",
                  f"\nData prevista para retirada: {row[4]}, Data efetiva da retirada: {row[5]}",
                  f"\nData da compra: {row[6]}",
                  f"\nData prevista da devolução: {row[7]}, Data efetiva da devolução: {row[8]}\n")

# 3.3. Relatório de fila de espera

def relatorio_fila_espera():
    
    # Verificando a presença de uma Marca válida
    while True:
        # Obtendo inputs
        marca_selecionada = input("Insira a marca a ser pesquisada (ex.: Toyota): ")

        # Verificar se a Marca existe
        if not verificar_marca_existe(marca_selecionada):
            print("\nNão exite um veículo com essa marca. Por favor, insira uma marca válida.")
            
        else: break
    
    # Verificando a presença de um Modelo válido
    while True:
        # Obtendo inputs
        modelo_selecionado = input("Insira o modelo da marca a ser pesquisado (ex.: Etios): ")

        # Verificar se a Placa existe
        if not verificar_modelo_existe(modelo_selecionado):
            print("\nNão exite um veículo com esse modelo. Por favor, insira uma modelo válido.")
            
        else: break
    
    # Consulta SQL
    query = """
    SELECT
        fl.Num_consul,
        fl.Data_consul,
        v.Marca,
        v.Modelo,
        sr.Num_client
    FROM
        fila_espera fl
    INNER JOIN subsis_reserv as sr
    ON fl.Num_consul = sr.Num_consul
    INNER JOIN veiculo as v
    ON sr.Placa = v.Placa
    WHERE v.Marca = %s AND v.Modelo = %s;
    """

    results = fetch_query(query, (marca_selecionada, modelo_selecionado))

    # Imprimir os resultados
    if results:
        for row in results:
            print(f"\nNúmero da consulta: {row[0]}, \nData da consulta: {row[1]}",
                  f"\nMarca do veículo: {row[2]}, \nModelo do veículo: {row[3]}",
                  f" \nNúmero de clientes na fila: {row[4]}\n")

In [4]:
# Definindo as funções a serem executadas pela aplicação

# Módulo 2: Realizar Consulta
    
# 1. Realizar Consulta de Frota
    
def consultar_frota():
    
    # Verificando a presença de uma Marca válida
    while True:
        # Obtendo inputs
        marca_selecionada = input("Insira a marca a ser pesquisada (ex.: Toyota): ")

        # Verificar se a Marca existe
        if not verificar_marca_existe(marca_selecionada):
            print("\nNão exite um veículo com essa marca. Por favor, insira uma marca válida.")
            
        else: break
    
    # Verificando a presença de um Modelo válido
    while True:
        # Obtendo inputs
        modelo_selecionado = input("Insira o modelo da marca a ser pesquisado (ex.: Etios): ")

        # Verificar se a Placa existe
        if not verificar_modelo_existe(modelo_selecionado):
            print("\nNão exite um veículo com esse modelo. Por favor, insira uma modelo válido.")
            
        else: break

    # Consulta SQL
    query = """
    SELECT Marca, Modelo, COUNT(Marca)
    FROM veiculo
    WHERE Marca = %s AND Modelo = %s
    GROUP BY Marca, Modelo
    """
    
    params = (marca_selecionada, modelo_selecionado)
    
    results = fetch_query(query, params)
    
    # Imprimir os resultados
    if results:
        for row in results:
            print(f"\nMarca: {row[0]}, \nModelo: {row[1]}, \nQuantidade de veículos: {row[2]}\n")

# 2. Acompanhamento de Contrato de Locação

# 2.1. Fechamento de locação

def fechamento_locacao():
    
    # Obtendo inputs
    Placa = input("Insira a placa do veículo (ex.: AAA1A11 ou AAA1111): ")
    
    if Placa and not verificar_placa_existe(Placa):
        print("\nA placa não existe, cadastre o veículo.")
        return
    
    Conta = input("Insira a conta do locatário (ex.:123456789-0: ")
    
    if Conta and not verificar_conta_existe(Conta):
        print("\nA conta não existe, cadastre o locatário.")
        return
    
    CPF = input("Insira o CPF do locatário (ex.:111.111.111-11): ")
    
    if CPF and not verificar_cpf_existe(CPF):
        print("\nO CPF não existe, cadastre a pessoa física.")
        return
    
    Seguros_ad = input("Insira 0. Não há seguros adicinoais \nou 1. Existem seguros adicionais: ")
    Data_retirada_prev = input("Insira a data prevista de retirada (AAAA-MM-DD HH:MM:SS): ")
    Data_retirada_efet = input("Insira a data efetiva de retirada (AAAA-MM-DD HH:MM:SS): ")
    Data_compra = input("Insira a data da compra (AAAA-MM-DD): ")
    Data_devol_prev = input("Insira a data prevista de devolução (AAAA-MM-DD HH:MM:SS): ")
    Data_devol_efet = input("Insira a data efetiva de devolução (AAAA-MM-DD HH:MM:SS): ")
    Patio_entra = input("Insira o pátio de entrada do veículo (ex.: Pátio A): ")
    Patio_saida = input("Insira o pátio de saída do veículo (ex.: Pátio B): ")
    
    query = """INSERT INTO subsis_locacao (Placa, Conta, CPF, Seguros_ad, Data_retirada_prev, 
                Data_retirada_efet, Data_compra, Data_devol_prev, Data_devol_efet, Patio_entra, Patio_saida) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    params = (Placa, Conta, CPF, Seguros_ad, Data_retirada_prev, Data_retirada_efet, Data_compra, 
              Data_devol_prev, Data_devol_efet, Patio_entra, Patio_saida)
    
    execute_query(query, params)
    print("\nNova locação realizada com sucesso!\n")

# 2.2. Cálculo de cobrança

def cobranca_inicial():
    
    while True:
        Num_reserv = input("Insira o número da reserva para o cálculo do valor previsto: ")

        # Verificar se a reserva existe
        if not verificar_reserva_existe(Num_reserv):
            print("\nNúmero da reserva não existe. Por favor, insira um número de reserva válido ou cadastre uma nova reserva.")
        else:
            break

    query = """
    SELECT
        l.Num_reserv,
        l.Placa,
        v.Modelo,
        l.CPF,
        l.Conta,
        l.Data_retirada_prev,
        l.Data_devol_prev,
        v.Preco,
        (DATEDIFF(l.Data_devol_prev, l.Data_retirada_prev)) AS Quantidade_dias_previstos,
        (DATEDIFF(l.Data_devol_prev, l.Data_retirada_prev) * v.Preco) AS Valor_locacao_prev
    FROM
        subsis_locacao l
    JOIN
        veiculo v ON l.Placa = v.Placa
    WHERE
        l.Num_reserv = %s
    """

    params = (Num_reserv,)

    results = fetch_query(query, params)
    
    # Imprimir os resultados
    if results:
        for row in results:
            print(f"\nNúmero da reserva: {row[0]}, \nPlaca do veículo: {row[1]}, \nModelo do veículo: {row[2]}",
                  f" \nCPF do locatário: {row[3]}, \nConta do locatário: {row[4]}, \nData de retirada prevista: {row[5]}",
                  f"\nData de devolução prevista: {row[6]}, \nPreço da diária do veículo: {row[7]}",
                  f"\nNúmero de dias previstos para a locação: {row[8]}, \nValor da locação: {row[9]}\n")

# 2.3 Controle de cobrança

def cobranca_final():
    
    while True:
        Num_reserv = input("Insira o número da reserva para o cálculo do valor efetivo: ")

        # Verificar se a reserva existe
        if not verificar_reserva_existe(Num_reserv):
            print("\nNúmero da reserva não existe. Por favor, insira um número de reserva válido ou cadastre uma nova reserva.")
        else:
            break

    # Verificando a necessidade de cobrança de multa por atraso

    # Primeira consulta para obter os dados de retirada e devolução previstas
    query_prev = """
    SELECT
        Data_retirada_prev,
        Data_devol_prev
    FROM
        subsis_locacao
    WHERE
        Num_reserv = %s
    """
    
    # Segunda consulta para obter os dados de retirada e devolução efetivas
    query_efet = """
    SELECT
        Data_retirada_efet,
        Data_devol_efet
    FROM
        subsis_locacao
    WHERE
        Num_reserv = %s
    """
    
    params = (Num_reserv,)
    data_prev = fetch_query(query_prev, params)
    data_efet = fetch_query(query_efet, params)
    
    if data_prev and data_efet:
        
        # Calculando as diferenças de datas
        data_retirada_prev, data_devol_prev = data_prev[0]
        data_retirada_efet, data_devol_efet = data_efet[0]

        dias_previstos = (data_devol_prev - data_retirada_prev).days
        dias_efetivos = (data_devol_efet - data_retirada_efet).days
        
        cobranca = 1 if dias_efetivos > dias_previstos else 0 # Comparando as diferenças para a cobrança ou não de multa:
    
        query = """
        SELECT
            l.Num_reserv,
            l.Placa,
            v.Modelo,
            l.CPF,
            l.Conta,
            l.Data_retirada_efet,
            l.Data_devol_efet,
            v.Preco,
            (DATEDIFF(l.Data_devol_efet, l.Data_retirada_efet)) AS Quantidade_dias_previstos,
            (DATEDIFF(l.Data_devol_efet, l.Data_retirada_efet) * v.Preco + 0.1 * DATEDIFF(l.Data_devol_efet, l.Data_retirada_efet) * v.Preco * %s) AS Valor_locacao_prev
        FROM
            subsis_locacao l
        JOIN
            veiculo v ON l.Placa = v.Placa
        WHERE
            l.Num_reserv = %s
        """
        
        params = (cobranca, Num_reserv)
        
        results = fetch_query(query, params)
        
        # Imprimir os resultados
        if results:
            for row in results:
                print(f"\nNúmero da reserva: {row[0]}, \nPlaca do veículo: {row[1]}, \nModelo do veículo: {row[2]}",
                      f" \nCPF do locatário: {row[3]}, \nConta do locatário: {row[4]}, \nData de retirada efetiva: {row[5]}",
                      f"\nData de devolução efetiva: {row[6]}, \nPreço da diária do veículo: {row[7]}",
                      f"\nNúmero de dias efetivos para a locação: {row[8]}",
                      f"\nDias de atraso: {(dias_efetivos - dias_previstos)*cobranca}",
                      f"\nValor da locação final (com multa de 10% se necessário): {row[9]}\n")

In [6]:
# Definindo as funções a serem executadas pela aplicação

# Módulo 3: Análise de resultados

def receita_mes_prevista():

    ano_consulta = input("Insira o ano da consulta do gráfico: (ex.: 2024)")
    mes_inicio = input("Insira o número do mês do início do gráfico (ex.: Janeiro = 1): ")
    mes_fim = input("Insira o número do mês do fim do gráfico (ex.: Julho = 7): ")

    query = """
    
    SELECT
        MONTH(l.Data_retirada_efet) AS mes,
        SUM(DATEDIFF(l.Data_devol_prev, l.Data_retirada_prev) * v.Preco) AS receita_prevista
    FROM
        subsis_locacao l
    JOIN
        veiculo v ON l.Placa = v.Placa
    WHERE
        MONTH(l.Data_retirada_efet) >= %s
        AND MONTH(l.Data_retirada_efet) <= %s
        AND YEAR(l.Data_retirada_efet) = %s
    GROUP BY 
        MONTH(l.Data_retirada_efet)
        """
    
    params = (mes_inicio, mes_fim, ano_consulta)
    results = fetch_query_grafs(query, params)
    
    # Extraindo os meses e receitas dos resultados
    meses = [row[0] for row in results]
    receitas_prevista = [row[1] for row in results]
    
    # Ordenando os resultados por ordem crescente de mês
    sorted_data = sorted(zip(meses, receitas_prevista))
    meses, receitas = zip(*sorted_data)
    
    # Plotando o gráfico
    plt.figure(figsize=(10, 5))
    plt.plot(meses, receitas_prevista, marker='o')
    plt.title('Receita por Mês')
    plt.xlabel('Mês')
    plt.ylabel('Receita')
    plt.xticks(meses)
    plt.grid(True)
    plt.show()

# Módulo 3: Sair

### Estrutura do menu

#### Módulo 1: Realizar Cadastro, Atualização ou Reserva

    1. Cadastro/Atualização de Clientes
    
        1.1 Cadastro de Cliente
        
            1.1.1 Cadastro de PF

            1.1.2 Cadastro de PJ
            
        1.3 Cadastro de Funcionário
        
        1.4 Cadastro de Locatario
        
        1.5 Cadastro de Condutor
        
    2. Cadastro/Atualização da Frota
        
        2.1 Cadastro de veículo
        
        2.2 Atualização do prontuário do veículo
        
        2.3 Cadastro de foto do veículo
    
    3. Reserva de Veículo

        3.1. Relatório de pátio

        3.2. Relatório de locação

        3.3. Relatório de fila de espera

#### Módulo 2: Realizar Consultas
    
    1. Realizar Consulta de Frota
    
    2. Acompanhamento de Contrato de Locação

        2.1. Fechamento de locação
        
        2.2. Cálculo de cobrança

        2.3. Controle de cobrança    

#### Módulo 3: Análises de resultado

    1. Gráfico de Receita Prevista por Mês
    
#### Móedulo 4: Sair

In [7]:
## Menu Principal
def menu():
    while True:
        print("\n1. Realizar Cadastro, Atualização ou Reserva")
        print("2. Realizar Consulta")
        print("3. Realizar Análise de Resultados")
        print("4. Sair")

        modulo = input("Escolha uma opção: ")

# 1. Realizar Cadastro, Atualização ou Reserva
        
        if modulo == "1":
            while True:

                print("\n1. Cadastro/Atualização de Clientes")
                print("2. Cadastro/Atualização da Frota")
                print("3. Reserva de Veículo")
                print("4. Sair de Cadastros, Atualizações ou Reservas")

                opcao = input("Escolha uma opção: ")

            # 1. Cadastro/Atualização de Clientes
                if opcao == "1":
                    print("\n1 Cadastro de Cliente")
                    print("2 Cadastro de Funcionário")
                    print("3 Cadastro de Locatario")
                    print("4 Cadastro de Condutor")

                    opcao1 = input("Escolha uma opção: ")

                    if opcao1 == "1":
                        print("\n1 Cadastro de Pessoa Física")
                        print("2 Cadastro de Pessoa Jurídica")

                        opcao1_1 = input("Escolha uma opção: ")

                        if opcao1_1 == "1":
                            cadastrar_cliente_pf()

                        elif opcao1_1 == "2":
                            cadastrar_cliente_pj()
                        
                    elif opcao1 == "2":
                        cadastrar_funcionario()

                    elif opcao1 == "3":
                        cadastrar_locatario()

                    elif opcao1 == "4":
                        cadastrar_condutor()

            # 2. Cadastro/Atualização da Frota

                elif opcao == "2":
                    print("\n1 Cadastro de veículo")
                    print("2 Atualização do prontuário do veículo")
                    print("3 Cadastro de foto do veículo")

                    opcao2 = input("Escolha uma opção: ")

                    if opcao2 == "1":
                        cadastrar_veiculo()

                    elif opcao2 == "2":
                        atualizar_cadastrar_prontuario()

                    elif opcao2 == "3":
                        cadastrar_foto_veiculo()

            # 3. Reserva de Veículo
                elif opcao == "3":
                    print("\n1. Relatório do pátio")
                    print("2. Relatório de locação")
                    print("3. Relatório de fila de espera")

                    opcao3 = input("Escolha uma opção: ")

                    if opcao3 == "1":
                        relatorio_patio()

                    elif opcao3 == "2":
                        relatorio_locacao()

                    elif opcao3 == "3":
                        relatorio_fila_espera()
                
                elif opcao == "4":
                    break

                    
# 2. Realizar Consultas

        elif modulo == "2":
            while True:

                print("\n1. Realizar Consulta de Frota")
                print("2. Fechamento de locação")
                print("3. Cálculo de cobrança")
                print("4. Controle de cobrança")
                print("5. Sair de Consultas")

                opcao = input("Escolha uma opção: ")

                if opcao == "1":
                    consultar_frota()

                elif opcao == "2":
                    fechamento_locacao()

                elif opcao == "3":
                    cobranca_inicial()

                elif opcao == "4":
                    cobranca_final()
                
                elif opcao == "5":
                    break


# 3. Análise de Resultados

        elif modulo == "3":
            while True:
                print("\n1. Realizar Consulta da receita por mês")
                print("2. Sair de Análise de Resultados")
                
                opcao = input("Escolha uma opção: ")

                if opcao == "1":
                    receita_mes_prevista()
                
                elif opcao == "2":
                    break
# 4. Sair
              
        elif modulo == "4":
            break

        else:
            print("Opção inválida!")

if __name__ == "__main__":
    menu()


1. Realizar Cadastro, Atualização ou Reserva
2. Realizar Consulta
3. Realizar Análise de Resultados
4. Sair
Escolha uma opção: 4
